In [1]:
import QuantLib as ql

# Testing Hagan-pricer flat-vol equivalence for coupons...

In [5]:
termStructure = ql.RelinkableYieldTermStructureHandle()

In [7]:
calendar = ql.TARGET()

In [8]:
referenceDate = calendar.adjust(ql.Date.todaysDate())

In [10]:
ql.Settings.instance().evaluationDate = referenceDate

In [11]:
termStructure.linkTo(ql.FlatForward(referenceDate, 0.05,
                                          ql.Actual365Fixed()))

In [12]:
iborIndex = ql.Euribor6M(termStructure)

In [13]:
swapIndex = ql.SwapIndex("EuriborSwapIsdaFixA",
                         ql.Period("10Y"),
                          iborIndex.fixingDays(),
                          iborIndex.currency(),
                          iborIndex.fixingCalendar(),
                          ql.Period("1Y"),
                          ql.Unadjusted,
                          iborIndex.dayCounter(),
                          iborIndex)

In [14]:
startDate = termStructure.referenceDate() + 20*ql.Years

In [16]:
paymentDate = startDate + 1*ql.Years

In [17]:
endDate = paymentDate

In [18]:
nominal = 1.0
infiniteCap = ql.nullDouble()
infiniteFloor = ql.nullDouble()
gearing = 1.0
spread = 0.0

In [20]:
coupon = ql.CappedFlooredCmsCoupon(paymentDate, nominal,
                                  startDate, endDate,
                                  swapIndex.fixingDays(), swapIndex,
                                  gearing, spread,
                                  infiniteCap, infiniteFloor,
                                  startDate, endDate,
                                  iborIndex.dayCounter())

In [24]:
yieldCurveModels = [ql.GFunctionFactory.Standard,
                   ql.GFunctionFactory.ExactYield,
                   ql.GFunctionFactory.ParallelShifts,
                   ql.GFunctionFactory.NonParallelShifts,
                   ql.GFunctionFactory.NonParallelShifts]

In [26]:
zeroMeanRev = ql.QuoteHandle(ql.SimpleQuote(0.0))

In [31]:
atmOptionTenors = [
    ql.Period(1, ql.Months),
    ql.Period(6, ql.Months),
    ql.Period(1, ql.Years),
    ql.Period(5, ql.Years),
    ql.Period(10, ql.Years),
    ql.Period(30, ql.Years)

]

In [32]:
atmSwapTenors = [
    ql.Period(1, ql.Years),
    ql.Period(5, ql.Years),
    ql.Period(10, ql.Years),
    ql.Period(30, ql.Years)
]

In [33]:
m = ql.Matrix(len(atmOptionTenors),len(atmSwapTenors))

In [34]:
m[0][0]=0.1300; m[0][1]=0.1560; m[0][2]=0.1390; m[0][3]=0.1220;
m[1][0]=0.1440; m[1][1]=0.1580; m[1][2]=0.1460; m[1][3]=0.1260;
m[2][0]=0.1600; m[2][1]=0.1590; m[2][2]=0.1470; m[2][3]=0.1290;
m[3][0]=0.1640; m[3][1]=0.1470; m[3][2]=0.1370; m[3][3]=0.1220;
m[4][0]=0.1400; m[4][1]=0.1300; m[4][2]=0.1250; m[4][3]=0.1100;
m[5][0]=0.1130; m[5][1]=0.1090; m[5][2]=0.1070; m[5][3]=0.0930;

In [39]:
atmVol = ql.SwaptionVolatilityStructureHandle(
            ql.SwaptionVolatilityMatrix(calendar,
                                       ql.Following,
                                             atmOptionTenors,
                                             atmSwapTenors,
                                             m,
                                             ql.Actual365Fixed()))

In [43]:
numericalPricers = [ql.NumericHaganPricer(atmVol, yieldCurveModels[j], zeroMeanRev) for j in range(0, len(yieldCurveModels)-1)]
numericalPricers += [ql.LinearTsrPricer(atmVol, zeroMeanRev)]
analyticPricers = [ql.AnalyticHaganPricer(atmVol, yieldCurveModels[j],
                                        zeroMeanRev) for j in range(0, len(yieldCurveModels))]

In [48]:
for j in range(0, len(yieldCurveModels)):
    numericalPricers[j].setSwaptionVolatility(atmVol)
    coupon.setPricer(numericalPricers[j])
    rate0 = coupon.rate()
    analyticPricers[j].setSwaptionVolatility(atmVol)
    coupon.setPricer(analyticPricers[j])
    rate1 = coupon.rate()
    difference =  abs(rate1-rate0)
    print(difference)

4.067231326732035e-07
2.3050488483222775e-08
5.235758604449359e-09
5.235758604449359e-09
4.2694381141572535e-08


# Testing Hagan-pricer flat-vol equivalence for swaps...

In [50]:
#tbd